In [65]:
import matplotlib.pyplot as plt
from random import randint
from sklearn.metrics import confusion_matrix
import numpy as np

In [66]:
%run 'Data_Munging.ipynb'

<class 'pandas.tslib.Timedelta'>
61.0


## 1) k-nearest neighbors classifier

On crée d'abord le testing set: 

In [67]:
d = 28
daySensor = [sensorData['Start time'][x].day == d and sensorData['End time'][x].day == d for x in range(1,len(sensorData))]
daySensor = daySensor + [False]
daySensorData = sensorData[daySensor]
daySensorData.index = np.arange(1,len(daySensorData)+1)

dayActivity = [activityData['Start time'][x].day == d and activityData['End time'][x].day == d for x in range(1,len(activityData))]
dayActivity = dayActivity + [False]
dayActivityData = activityData[dayActivity]
dayActivityData.index = np.arange(1,len(dayActivityData)+1)

testingFeatureMatrix, testingLabel = convert2LastFiredFeatureMatrix(daySensorData,dayActivityData,60)

cumuSen, cumuAct = cumulationTable(testingFeatureMatrix, testingLabel)

sensorTestingData = np.asarray(cumuSen)
labelTestingData = np.asarray(cumuAct)

In [68]:
from sklearn.neighbors import KNeighborsClassifier

In [69]:
sensorTrainingData = np.asarray(cumuSensor)
labelTrainingData = np.asarray(cumuActivity)

In [70]:
knn = KNeighborsClassifier()#(n_neighbors=3, weights='distance')
knn.fit(sensorTrainingData, labelTrainingData)

predictLabel = knn.predict(sensorTestingData)

In [71]:
error_rate = 0.0
for i in xrange(len(predictLabel)):
    if predictLabel[i] != labelTestingData[i]: 
        error_rate += 1
print error_rate/len(predictLabel)

0.15


In [72]:
predictLabel

array([ 4,  0, 10,  4,  0, 17, 13,  0,  4,  5,  5,  1,  0,  4,  0,  1,  0,
        4,  0,  4])

In [ ]:
confusion_matrix(predictLabel, labelTestingData)

array([[6, 0, 0, 0, 1, 0, 0],
       [0, 2, 0, 0, 0, 0, 0],
       [0, 0, 6, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0]])

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    #tick_marks = np.arange(len(set(labelTestingData)))
    #plt.xticks(tick_marks,list(set(labelTestingData)), rotation=45)
    #plt.yticks(tick_marks, list(set(labelTestingData)))
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# Compute confusion matrix
cm = confusion_matrix(labelTestingData, predictLabel)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[6 0 0 1 0 0 0]
 [0 2 0 0 0 0 0]
 [0 0 6 0 0 0 1]
 [0 0 0 1 0 0 0]
 [1 0 0 0 1 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0]]
Normalized confusion matrix